In [30]:
!git clone https://github.com/Samudragupta123/sentiment_project.git

fatal: destination path 'sentiment_project' already exists and is not an empty directory.


In [31]:
import os
print(os.listdir())

['.config', 'sentiment_project', 'sample_data']


In [32]:
print(os.listdir('sentiment_project/data'))

['raw_dataset.csv']


In [33]:
import pandas as pd
import numpy as np
dataset=pd.read_csv('sentiment_project/data/raw_dataset.csv')
dataset.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [34]:
#dropping redundant columns
dataset=dataset.drop(dataset.columns[[0,6,7,8,9,11,12,13,14]], axis=1)

In [35]:
dataset.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,text
0,neutral,1.0000,NaN,NaN,Virgin America,@VirginAmerica What @dhepburn said.
1,positive,0.3486,NaN,0.0000,Virgin America,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,NaN,NaN,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,Bad Flight,0.7033,Virgin America,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,Can't Tell,1.0000,Virgin America,@VirginAmerica and it's a really big bad thing...


In [36]:
datasetfinal=dataset.fillna(0)

In [37]:
datasetfinal.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,text
0,neutral,1.0000,0,0.0000,Virgin America,@VirginAmerica What @dhepburn said.
1,positive,0.3486,0,0.0000,Virgin America,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,0,0.0000,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,Bad Flight,0.7033,Virgin America,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,Can't Tell,1.0000,Virgin America,@VirginAmerica and it's a really big bad thing...


In [38]:
X=dataset.iloc[:,1:5].values
y=dataset.iloc[:,0].values

In [39]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [42]:
import torch
import numpy as np
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.y_data)

    def __getitem__(self, idx):
        # Get one sample from the numpy arrays
        sample_features = self.x_data[idx]
        sample_label = self.y_data[idx]
train_dataset= MyDataset(X_train, y_train)
test_dataset= MyDataset(X_test, y_test)

In [43]:
from torch.utils.data import DataLoader

# Training loader: Shuffling is True
train_loader = DataLoader(
    dataset=train_dataset, 
    batch_size=1000, 
    shuffle=True  
)

# Testing/Validation loader: Shuffling is usually False
test_loader = DataLoader(
    dataset=test_dataset, 
    batch_size=100, 
    shuffle=False
)
